In [1]:
from dataclasses import dataclass, field
import copy
from enum import Enum, auto
import random
import pandas as pd

In [2]:
# cards are numbers from 1 to 13
# the score is capped at 10

## Define a Hand class and functions on it

In [3]:
@dataclass
class Hand:
    """Class for representing a blackjack hand."""
    score: int = 0
    soft: bool = False
    cards: [int] = field(default_factory=list)
    doubled: bool = False


In [4]:
def add_card(hand, card):
    if card != 1:  
        if (not hand.soft) or (hand.score <= 11): # simple case
            new_score = min(hand.score+min(10, card), 22) # cap busted hands at 22
        else: # make a soft hand hard
            new_score = hand.score+min(10, card) - 10 
            hand.soft = False
    else: # card is an ace
        if hand.score >= 11: # 11s and up count an ace as 1 (hard or soft)
            new_score = min(hand.score+min(10, card), 22) # cap busted hands at 22
        else: # soft ace
            new_score = hand.score+11
            hand.soft = True

    hand.score = new_score
    hand.cards += [card]
    return hand

In [5]:
def make_hand(cards):
    h = Hand()
    for c in cards:
        h = add_card(h, c)
    return h

make_hand([4,6])

Hand(score=10, soft=False, cards=[4, 6], doubled=False)

In [6]:
def is_busted(hand):
    return hand.score > 21

In [7]:
def is_blackjack(hand):
    return hand.score==21 and len(hand.cards)==2

In [8]:
h = Hand()
h

Hand(score=0, soft=False, cards=[], doubled=False)

In [9]:
add_card(h, 6) # start a hand with a 6

Hand(score=6, soft=False, cards=[6], doubled=False)

In [10]:
add_card(h, 11) # show that J (11) counts as 10 points

Hand(score=16, soft=False, cards=[6, 11], doubled=False)

In [11]:
add_card(h, 7), is_busted(h) # bust (show that 23 is counted as 22)

(Hand(score=22, soft=False, cards=[6, 11, 7], doubled=False), True)

In [12]:
# OK, now let's try another hand with aces
h = Hand()
add_card(h, 1) # should be a soft 11

Hand(score=11, soft=True, cards=[1], doubled=False)

In [13]:
add_card(h, 1) # two aces: should be a soft 12

Hand(score=12, soft=True, cards=[1, 1], doubled=False)

In [14]:
add_card(h, 11) # two aces and a J: should be a hard 12

Hand(score=12, soft=False, cards=[1, 1, 11], doubled=False)

In [15]:
# OK, now let's try another hand with a blackjack
h = Hand()
add_card(h, 1) # should be a soft 11
add_card(h, 10)
h, is_blackjack(h)

(Hand(score=21, soft=True, cards=[1, 10], doubled=False), True)

## Now define gameplay and strategy

In [16]:

# TODO I might want a Flag class later, to provide a set of possible Actions
class Action(Enum):
    STAND = auto()
    HIT = auto()
    DOUBLE = auto()
    #SPLIT = auto()
    
    

In [17]:
# Most simple/conservative strategy imaginable:
def strat_nobust(hand, dealer):
    if hand.score > 11:
        return Action.STAND
    else:
        return Action.HIT
        
strat_nobust.name = 'strat_nobust'

In [18]:
# Dealer strategy
def strat_dealer(hand, dealer):
    if hand.score < 17:
        return Action.HIT
    # TODO handle soft hands
    else:
        return Action.STAND
    
strat_dealer.name='strat_dealer'
        

In [19]:
class HandOutcome(Enum):
    WIN = 1
    LOSE = -1
    WIN_DOUBLE = 2
    LOSE_DOUBLE = -2
    PUSH = 0
    BLACKJACK = 1.5

In [20]:
# Deck; completely random (i.e., infinite) for now


def deal_card():
    return random.randrange(13)+1

In [21]:
[deal_card() for _ in range(10)]

[10, 13, 13, 10, 4, 3, 1, 3, 8, 3]

In [22]:
# return the final hand after playing
def player_play_hand(strategy, hand, dealer, deck):
    while True:
        decision = strategy(hand, dealer)
        if decision == Action.STAND:
            return hand
        if decision == Action.HIT:
            add_card(hand, deck())
            if is_busted(hand):
                return hand
        if decision == Action.DOUBLE:
            hand.doubled = True
            add_card(hand, deck())
            return hand



In [23]:
def player_hand_outcome(player_hand, dealer_hand):
    # First compute the initial outcome, then double it if necessary for a double-down
    def initial_outcome():
        if is_blackjack(player_hand):
            if is_blackjack(dealer_hand):
                return HandOutcome.PUSH
            else:
                return HandOutcome.BLACKJACK
        if is_busted(player_hand) or is_blackjack(dealer_hand):
            return HandOutcome.LOSE
        if is_busted(dealer_hand):
            return HandOutcome.WIN
        if player_hand.score > dealer_hand.score:
            return HandOutcome.WIN
        if player_hand.score == dealer_hand.score:
            return HandOutcome.PUSH
        if player_hand.score < dealer_hand.score:
            return HandOutcome.LOSE

    outcome = initial_outcome()

    outcome_doubler = {HandOutcome.WIN: HandOutcome.WIN_DOUBLE, HandOutcome.LOSE: HandOutcome.LOSE_DOUBLE}

    if player_hand.doubled:
        outcome = outcome_doubler.get(outcome) or outcome
    return outcome
        


In [24]:
def get_strat_name(strat):
    if hasattr(strat, 'name'):
        return strat.name
    return repr(strat)

In [25]:
# Goal is to evaluate strategies, so make comparisons simple

# For each round:
# Multiple players all play with a copy of the same starting hand
# Each player has a strategy that they play
# Dealer plays dealer strategy

# For now, we're using an infinite deck and strategies without knowledge, so
# the interaction of players/strategies should be a wash

def deal_one_round():
    hand_p = Hand()
    hand_d = Hand()

    add_card(hand_p, deal_card())
    add_card(hand_d, deal_card())
    add_card(hand_p, deal_card())
    
    dealer_hole_card = deal_card()
    
    return hand_p, hand_d, dealer_hole_card


# Play multiple strategies on one starting point
def complete_one_round(strats, player_hand, dealer_hand, dealer_hole_card):
    hand_p = copy.deepcopy(player_hand)
    hand_d = copy.deepcopy(dealer_hand)
    
    # represent each player as a hand and a strategy
    players = [(player_play_hand(strat, copy.deepcopy(hand_p), hand_d, deal_card), get_strat_name(strat)) for strat in strats]
    
    # dealer
    player_play_hand(strat_dealer, add_card(hand_d, dealer_hole_card), None, deal_card)
    
    return [(strat, hand_p, hand_d, player_hand_outcome(hand_p, hand_d)) for (hand_p, strat) in players]

    
def play_one_round(strats):
    hand_p, hand_d, dealer_hole_card = deal_one_round()
    return complete_one_round(strats, hand_p, hand_d, dealer_hole_card)

play_one_round([strat_nobust, strat_nobust])

[('strat_nobust',
  Hand(score=17, soft=False, cards=[8, 9], doubled=False),
  Hand(score=17, soft=False, cards=[8, 9], doubled=False),
  <HandOutcome.PUSH: 0>),
 ('strat_nobust',
  Hand(score=17, soft=False, cards=[8, 9], doubled=False),
  Hand(score=17, soft=False, cards=[8, 9], doubled=False),
  <HandOutcome.PUSH: 0>)]

## Aggregate and summarize the data from the simulations

In [26]:

def generate_row_from_player(player):
    (strat, hand_p, hand_d, outcome) = player
    return {'strategy': strat, 'hand_start': hand_p.cards[:2], 'dealer_card': hand_d.cards[0], 'hand_end': hand_p.cards, 'dealer_hand': hand_d.cards, 'outcome': outcome}

def generate_rows_from_round(r):
    return [generate_row_from_player(player) for player in r]


generate_rows_from_round(play_one_round([strat_nobust, strat_dealer]))


[{'strategy': 'strat_nobust',
  'hand_start': [4, 7],
  'dealer_card': 13,
  'hand_end': [4, 7, 5],
  'dealer_hand': [13, 12],
  'outcome': <HandOutcome.LOSE: -1>},
 {'strategy': 'strat_dealer',
  'hand_start': [4, 7],
  'dealer_card': 13,
  'hand_end': [4, 7, 2, 7],
  'dealer_hand': [13, 12],
  'outcome': <HandOutcome.PUSH: 0>}]

In [27]:
def run_n_sim_trials(strats, n):
    sims = [generate_rows_from_round(play_one_round(strats)) for _ in range(n)]
    results = pd.DataFrame([player for round in sims for player in round])
    results['outcome_value'] = results['outcome'].apply(lambda x: x.value)
    results['outcome_name'] = results['outcome'].apply(lambda x: str(x)[12:])
    return results

def summarize_totals(sims):
    def outcome_name(x): return x.head(1) # The function name will be used as the column name
    outcome_counts = sims.groupby(['strategy', 'outcome_value'])['outcome_name'].agg([len, outcome_name])
    outcome_summary = outcome_counts.reset_index().set_index('strategy').drop(['outcome_value'], axis=1).pivot(columns=['outcome_name'])

    # The empty cells are NaNs; fill the NaNs and convert back to int
    for col in outcome_summary.columns:
        outcome_summary[col] = outcome_summary[col].fillna(0).apply(int)
        
    outcome_summary['mean_outcome'] = sims.groupby('strategy')['outcome_value'].mean()
    
    return outcome_summary


sim_results = run_n_sim_trials([strat_nobust, strat_dealer], 1000)
sim_results, summarize_totals(sim_results)

(          strategy hand_start  dealer_card    hand_end         dealer_hand  \
 0     strat_nobust     [8, 1]            6      [8, 1]          [6, 7, 12]   
 1     strat_dealer     [8, 1]            6      [8, 1]          [6, 7, 12]   
 2     strat_nobust     [9, 6]            8      [9, 6]          [8, 8, 12]   
 3     strat_dealer     [9, 6]            8  [9, 6, 10]          [8, 8, 12]   
 4     strat_nobust    [10, 6]            6     [10, 6]     [6, 2, 5, 3, 3]   
 ...            ...        ...          ...         ...                 ...   
 1995  strat_dealer     [6, 8]            1   [6, 8, 7]  [1, 5, 2, 2, 5, 4]   
 1996  strat_nobust    [9, 12]           13     [9, 12]         [13, 4, 11]   
 1997  strat_dealer    [9, 12]           13     [9, 12]         [13, 4, 11]   
 1998  strat_nobust    [11, 7]            4     [11, 7]        [4, 7, 3, 5]   
 1999  strat_dealer    [11, 7]            4     [11, 7]        [4, 7, 3, 5]   
 
                outcome  outcome_value outcome_nam

In [28]:
def strat_simple(hand, dealer):
    if hand.score == 11:  return Action.DOUBLE
    if hand.score >= 17:  return Action.STAND
    if hand.score <= 11:  return Action.HIT
    if dealer.score in (range(3,7)):  return Action.STAND
    else:  return Action.HIT
        
strat_simple.name = 'simple'


sims = run_n_sim_trials([strat_simple], 1000)
sims.head(10), summarize_totals(sims)

(  strategy hand_start  dealer_card       hand_end    dealer_hand  \
 0   simple    [13, 1]           12        [13, 1]     [12, 2, 8]   
 1   simple    [4, 12]            9    [4, 12, 13]         [9, 1]   
 2   simple     [1, 6]            4         [1, 6]     [4, 12, 9]   
 3   simple   [12, 11]           11       [12, 11]       [11, 12]   
 4   simple     [5, 3]           12      [5, 3, 1]  [12, 3, 3, 6]   
 5   simple    [11, 7]            5        [11, 7]     [5, 8, 11]   
 6   simple     [8, 9]           11         [8, 9]       [11, 13]   
 7   simple     [4, 8]            2     [4, 8, 11]     [2, 9, 10]   
 8   simple     [2, 1]            5         [2, 1]     [5, 10, 4]   
 9   simple     [2, 1]            7  [2, 1, 9, 13]      [7, 8, 6]   
 
                  outcome  outcome_value outcome_name  
 0  HandOutcome.BLACKJACK            1.5    BLACKJACK  
 1       HandOutcome.LOSE           -1.0         LOSE  
 2        HandOutcome.WIN            1.0          WIN  
 3       HandOu

In [29]:
summarize_totals(run_n_sim_trials([strat_dealer, strat_nobust, strat_simple], 10000))


len                                         mean_outcome
outcome_name BLACKJACK  LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                                   
simple             452  4655         224  862  3482        325      -0.0293
strat_dealer       452  5003           0  953  3592          0      -0.0733
strat_nobust       452  5221           0  601  3726          0      -0.0817

## Simulate specific situations to determine strategy

In [30]:
# 12 vs deuce

hand_p = make_hand([5, 7])
hand_d = make_hand([2])

hand_p, hand_d

(Hand(score=12, soft=False, cards=[5, 7], doubled=False),
 Hand(score=2, soft=False, cards=[2], doubled=False))

In [31]:
# Implement a strategy that composes conditions and actions

def cond_12_2(hand, dealer):
    return hand.score == 12 and dealer.score == 2 and not hand.soft
conditions = [(cond_12_2, Action.STAND)]

def generate_strat_conditional(strat_base, conditions):
    def strat_cond(hand, dealer):
        for (condition, action) in conditions:
            if condition(hand, dealer): return action
        return strat_base(hand, dealer)
    strat_cond.name = 'strat_cond'
    return strat_cond
    
strat_cond = generate_strat_conditional(strat_simple, conditions)

In [32]:
def gen_cond_strategies(strat_base, condition, actions):
    def gen_strat_action(strat_base, condition, action):
        strat = generate_strat_conditional(strat_base, [(condition, action)])
        strat.name = repr(action)
        return strat
    
    strats = [gen_strat_action(strat_base, condition, a) for a in actions]
    return strats
    



strats = gen_cond_strategies(strat_simple, cond_12_2, [Action.HIT, Action.STAND, Action.DOUBLE])
strats

[<function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>]

In [33]:
complete_one_round(strats, hand_p, hand_d, deal_card())

[('<Action.HIT: 2>',
  Hand(score=22, soft=False, cards=[5, 7, 4, 10], doubled=False),
  Hand(score=20, soft=False, cards=[2, 4, 4, 4, 6], doubled=False),
  <HandOutcome.LOSE: -1>),
 ('<Action.STAND: 1>',
  Hand(score=12, soft=False, cards=[5, 7], doubled=False),
  Hand(score=20, soft=False, cards=[2, 4, 4, 4, 6], doubled=False),
  <HandOutcome.LOSE: -1>),
 ('<Action.DOUBLE: 3>',
  Hand(score=13, soft=False, cards=[5, 7, 1], doubled=True),
  Hand(score=20, soft=False, cards=[2, 4, 4, 4, 6], doubled=False),
  <HandOutcome.LOSE_DOUBLE: -2>)]

In [34]:
# TODO reduce duplication with run_n_sim_trials
def run_n_sim_trials_from_state(strats, hand_p, hand_d, n):
    sims = [generate_rows_from_round(complete_one_round(strats, hand_p, hand_d, deal_card())) for _ in range(n)]
    results = pd.DataFrame([player for round in sims for player in round])
    results['outcome_value'] = results['outcome'].apply(lambda x: x.value)
    results['outcome_name'] = results['outcome'].apply(lambda x: str(x)[12:])
    return results

sims = run_n_sim_trials_from_state(strats, hand_p, hand_d, 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        5955  474     0       3571      -0.4768
<Action.HIT: 2>     6002           0  636  3362          0      -0.2640
<Action.STAND: 1>   6312           0    0  3688          0      -0.2624

In [35]:
# Soft 12 vs 2
hand_12s = make_hand([1, 1])

hand_12s, hand_d

(Hand(score=12, soft=True, cards=[1, 1], doubled=False),
 Hand(score=2, soft=False, cards=[2], doubled=False))

In [36]:
def cond_12s_2(hand, dealer):
    return hand.score == 12 and dealer.score == 2 and hand.soft

strats = gen_cond_strategies(strat_simple, cond_12s_2, [Action.HIT, Action.STAND, Action.DOUBLE])
strats

[<function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>]

In [37]:
sims = run_n_sim_trials_from_state(strats+[strat_simple], hand_12s, hand_d, 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        6299    0     0       3701      -0.5196
<Action.HIT: 2>     4780         404  812  3325        679      -0.0905
<Action.STAND: 1>   6299           0    0  3701          0      -0.2598
simple              4760         443  760  3291        746      -0.0863

In [38]:
summarize_totals(run_n_sim_trials_from_state([strat_simple, strat_cond], hand_p, hand_d, 100*1000))

len              mean_outcome
outcome_name   LOSE  PUSH    WIN             
strategy                                     
simple        61158  6503  32339     -0.28819
strat_cond    63595     0  36405     -0.27190

In [39]:
# soft 13 vs 6
def cond_13s_6(hand, dealer):
    return hand.score == 13 and dealer.score == 6 and hand.soft



strats = gen_cond_strategies(strat_simple, cond_13s_6, [Action.HIT, Action.STAND, Action.DOUBLE])
sims = run_n_sim_trials_from_state(strats+[strat_simple], make_hand([1, 2]), make_hand([6]), 100000)
summarize_totals(sims)

len                                     mean_outcome
outcome_name         LOSE LOSE_DOUBLE  PUSH    WIN WIN_DOUBLE             
strategy                                                                  
<Action.DOUBLE: 3>      0       57993     0      0      42007     -0.31972
<Action.HIT: 2>     45447        3282  3103  41406       6762      0.02919
<Action.STAND: 1>   57993           0     0  42007          0     -0.15986
simple              57993           0     0  42007          0     -0.15986

In [40]:
# 10 v 6
def cond_10_6(hand, dealer):
    return hand.score == 10 and dealer.score == 6 and not hand.soft

strats = gen_cond_strategies(strat_simple, cond_10_6, [Action.HIT, Action.STAND, Action.DOUBLE])
sims = run_n_sim_trials_from_state(strats+[strat_simple], make_hand([5, 5]), make_hand([6]), 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        3249  667     0       6084       0.5670
<Action.HIT: 2>     3270           0  698  6032          0       0.2762
<Action.STAND: 1>   5805           0    0  4195          0      -0.1610
simple              3267           0  702  6031          0       0.2764

In [41]:
ACTIONS = [Action.HIT, Action.STAND, Action.DOUBLE]

def test_cond(score_p, score_d, n):
    def cond(h, d):
        return h.score == score_p and d.score == score_d
    strats = gen_cond_strategies(strat_simple, cond, ACTIONS)
    hand = make_hand([8, score_p-8]) # This will only work for 10 through 18 
    sims = run_n_sim_trials_from_state(strats, hand, make_hand([score_d]), n)
    print(summarize_totals(sims))
    return cond, summarize_totals(sims)

test_cond(10, 6, 200)

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0          70   13    0        117        0.470
<Action.HIT: 2>      57           0   13  130          0        0.365
<Action.STAND: 1>   118           0    0   82          0       -0.180


(<function __main__.test_cond.<locals>.cond(h, d)>,
                     len                                  mean_outcome
 outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
 strategy                                                             
 <Action.DOUBLE: 3>    0          70   13    0        117        0.470
 <Action.HIT: 2>      57           0   13  130          0        0.365
 <Action.STAND: 1>   118           0    0   82          0       -0.180)

In [42]:
def find_winning_action(score_p, score_d, n):
    cond, summary = test_cond(score_p, score_d, n)
    outcomes = summary['mean_outcome']
    winner = outcomes[outcomes==outcomes.max()].index[0]
    winning_act = [a for a in ACTIONS if repr(a)==winner][0]
    return (cond, winning_act)

find_winning_action(10, 6, 200)

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0          65   14    0        121        0.560
<Action.HIT: 2>      72           0   19  109          0        0.185
<Action.STAND: 1>   121           0    0   79          0       -0.210


(<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>)

In [43]:
conds = [find_winning_action(p, d, 1000) for p in range(10, 19) for d in range(1, 11)]

                    len           mean_outcome
outcome_name       LOSE PUSH  WIN             
strategy                                      
<Action.DOUBLE: 3>  557   90  353       -0.204
<Action.HIT: 2>     575   79  346       -0.229
<Action.STAND: 1>   545   89  366       -0.179
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         340   83    0        577        0.474
<Action.HIT: 2>     356           0   91  553          0        0.197
<Action.STAND: 1>   633           0    0  367          0       -0.266
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         370   76    0        554        0.368
<Action.HIT: 2>   

                    len           mean_outcome
outcome_name       LOSE PUSH  WIN             
strategy                                      
<Action.DOUBLE: 3>  755   59  186       -0.569
<Action.HIT: 2>     748   47  205       -0.543
<Action.STAND: 1>   747   52  201       -0.546
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         609   40    0        351       -0.516
<Action.HIT: 2>     609           0   71  320          0       -0.289
<Action.STAND: 1>   646           0    0  354          0       -0.292
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         600   46    0        354       -0.492
<Action.HIT: 2>   

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         671   56    0        273       -0.796
<Action.HIT: 2>     657           0   68  275          0       -0.382
<Action.STAND: 1>   654           0    0  346          0       -0.308
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         637   51    0        312       -0.650
<Action.HIT: 2>     636           0   53  311          0       -0.325
<Action.STAND: 1>   611           0    0  389          0       -0.222
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         704   43    0        253       -0.902
<Action.HIT: 2>     710           0   51  239          0       -0.471
<Action.STAND: 1>   616           0    0  384          0       -0.232
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         729   36    0        235       -0.988
<Action.HIT: 2>     677           0   50  273          0       -0.404
<Action.STAND: 1>   594           0    0  406          0       -0.188
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         801   27    0        172       -1.258
<Action.HIT: 2>     784           0   19  197          0       -0.587
<Action.STAND: 1>   358           0  126  516          0        0.158
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         801   22    0        177       -1.248
<Action.HIT: 2>     781           0   24  195          0       -0.586
<Action.STAND: 1>   316           0  127  557          0        0.241
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

In [44]:
conds

[(<function __main__.test_cond.<locals>.cond(h, d)>, <Action.STAND: 1>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.HIT: 2>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.STAND: 1>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.con

In [45]:
strat_derived = generate_strat_conditional(strat_simple, conds)
strat_derived.name = 'strat_derived'

In [46]:
strat_derived2 = generate_strat_conditional(strat_derived, conds)
strat_derived2.name = 'strat_derived2'

In [47]:
sims = run_n_sim_trials([strat_simple, strat_derived, strat_derived2], 100*1000)
sims.head(10), summarize_totals(sims)

(         strategy hand_start  dealer_card          hand_end dealer_hand  \
 0          simple    [12, 8]            7           [12, 8]     [7, 10]   
 1   strat_derived    [12, 8]            7           [12, 8]     [7, 10]   
 2  strat_derived2    [12, 8]            7           [12, 8]     [7, 10]   
 3          simple    [4, 10]            6           [4, 10]  [6, 13, 1]   
 4   strat_derived    [4, 10]            6           [4, 10]  [6, 13, 1]   
 5  strat_derived2    [4, 10]            6           [4, 10]  [6, 13, 1]   
 6          simple     [1, 4]           13  [1, 4, 1, 11, 1]     [13, 9]   
 7   strat_derived     [1, 4]           13     [1, 4, 10, 3]     [13, 9]   
 8  strat_derived2     [1, 4]           13      [1, 4, 4, 9]     [13, 9]   
 9          simple     [4, 6]           10        [4, 6, 11]     [10, 8]   
 
             outcome  outcome_value outcome_name  
 0   HandOutcome.WIN            1.0          WIN  
 1   HandOutcome.WIN            1.0          WIN  
 2   Hand